In [1]:
!pip install azureml-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.4/901.4 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

# Use interactive login for Azure authentication (without service principal)
auth = InteractiveLoginAuthentication()

# Replace with your correct subscription, resource group, and workspace name
ws = Workspace.get(name='travel_bot',
                   subscription_id='6348e0cb-ef98-4058-9cf9-449e12a46b7a',
                   resource_group='travel-bot',
                   auth=auth)

print(f'Workspace {ws.name} loaded successfully.')


Performing interactive authentication. Please follow the instructions on the terminal.


Interactive authentication successfully completed.
Workspace travel_bot loaded successfully.


In [ ]:
from azureml.core import Model

# Register your models (replace with your actual file paths)
encoder_model = Model.register(model_path="/content/scripted_encoder.pt",
                                model_name="scripted_encoder",
                                workspace=ws)

decoder_model = Model.register(model_path="/content/scripted_decoder.pt",
                                model_name="scripted_decoder",
                                workspace=ws)

config_model = Model.register(model_path="/content/model_config.pth",
                               model_name="model_config",
                               workspace=ws)

print("Models registered successfully")


In [4]:
from azureml.core import Workspace, Model, Environment
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig


# Load the registered models
encoder_model = Model(ws, name='scripted_encoder')
decoder_model = Model(ws, name='scripted_decoder')
config_model = Model(ws, name='model_config3')

print("Models loaded successfully.")


Models loaded successfully.


In [5]:
from azureml.core import Environment

# List all environments available in your workspace
envs = Environment.list(workspace=ws)
for env in envs:
    print(env)


AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu


In [6]:
 # Define the inference configuration using the Azure managed PyTorch environment
 inference_config = InferenceConfig(
    entry_script='score.py',
    environment=Environment.get(ws, name='AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu')
 )




'''
# Create an environment using the env.yml file
env = Environment.from_conda_specification(
    name="my-custom-env",
    file_path="env.yml"
)

# Define the inference configuration using your custom environment
inference_config = InferenceConfig(
    entry_script='score.py',
    environment=env
)
'''

In [7]:
from azureml.core.webservice import AciWebservice

# Define ACI deployment configuration (adjust CPU and memory as needed)
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=2,
    auth_enabled=True
)

# Deploy the service using ACI
service_name = 'german-translate'
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[encoder_model, decoder_model, config_model],
    inference_config=inference_config,
    deployment_config=aci_config
)

# Wait for deployment to complete
service.wait_for_deployment(show_output=True)

# Check the status
print(f"Service state: {service.state}")
print(f"Scoring URI: {service.scoring_uri}")
print(f"Swagger URI: {service.swagger_uri}")

<ipython-input-7-aaacff327aa2>:12: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-11-14 14:32:23+00:00 Creating Container Registry if not exists.
2024-11-14 14:32:24+00:00 Use the existing image.
2024-11-14 14:32:27+00:00 Submitting deployment to compute.
2024-11-14 14:32:37+00:00 Checking the status of deployment german-translate..
2024-11-14 14:39:55+00:00 Checking the status of inference endpoint german-translate.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state: Healthy
Scoring URI: http://f0bcf2b8-a015-46f7-a2f2-ff3edf469c41.centralindia.azurecontainer.io/score
Swagger URI: http://f0bcf2b8-a015-46f7-a2f2-ff3edf469c41.centralindia.azurecontainer.io/swagger.json


In [8]:
!pip install torchtext==0.15.2 torch==2.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 848.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [9]:
import json
import torch
import os

# Constants for special tokens
BOS_IDX = 2  # Beginning of Sentence
EOS_IDX = 3  # End of Sentence
PAD_IDX = 1  # Padding

# Initialize global variables
scripted_encoder = None
scripted_decoder = None
vocab_de = None
vocab_en = None
de_tokenizer = None
en_tokenizer = None

def load_model_and_config():
    """
    Load the models and configuration data directly for Kaggle environment.
    """
    global scripted_encoder, scripted_decoder, vocab_de, vocab_en, de_tokenizer, en_tokenizer

    try:
        # Load the TorchScript-ed encoder and decoder models independently
        try:
            scripted_encoder = torch.jit.load('/content/scripted_encoder.pt', map_location='cpu')
            print("Encoder model loaded successfully.")
        except Exception as e:
            print(f"Error loading encoder model: {e}")

        try:
            scripted_decoder = torch.jit.load('/content/scripted_decoder.pt', map_location='cpu')
            print("Decoder model loaded successfully.")
        except Exception as e:
            print(f"Error loading decoder model: {e}")

        # Load the configuration data
        try:
            config_data = torch.load('/content/model_config3.pth', map_location='cpu')
            print("Configuration data loaded successfully.")
        except Exception as e:
            print(f"Error loading configuration: {e}")

        # Extract vocabularies and tokenizers
        if config_data:
            vocab_de = config_data.get('vocab_de', None)
            vocab_en = config_data.get('vocab_en', None)
            de_tokenizer = config_data.get('de_tokenizer', None)
            en_tokenizer = config_data.get('en_tokenizer', None)

        # Initialize tokenizers if not provided
        if de_tokenizer is None:
            de_tokenizer = lambda sentence: sentence.split()  # Simple tokenizer for German (split by space)
        if en_tokenizer is None:
            en_tokenizer = lambda sentence: sentence.split()  # Simple tokenizer for English (split by space)

        print("Model and configurations loaded successfully!")

    except Exception as e:
        print(f"Error during initialization: {str(e)}")


def translate_sentence(sentence, max_len=50, device='cpu'):
    """
    Translate a sentence from German to English using the trained models.
    """
    if scripted_encoder is None or scripted_decoder is None:
        print("Error: Models are not loaded.")
        return None

    scripted_encoder.eval()
    scripted_decoder.eval()

    # Tokenize and convert the input sentence in German to indices
    tokens = de_tokenizer(sentence.rstrip("\n"))  # Tokenize the German sentence
    src_ids = [vocab_de[token] for token in tokens] + [EOS_IDX]  # Convert tokens to indices and append <EOS>
    src_tensor = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)

    # Pass the source sentence through the encoder
    with torch.no_grad():
        encoder_outputs, encoder_hidden = scripted_encoder(src_tensor)

    tgt_ids = [BOS_IDX]  # Decoder input starts with <BOS>
    tgt_tensor = torch.tensor([tgt_ids], dtype=torch.long).to(device)
    decoder_hidden = encoder_hidden

    output_tokens = []

    for _ in range(max_len):
        with torch.no_grad():
            y, decoder_hidden = scripted_decoder(tgt_tensor, decoder_hidden)
            next_token_id = y.argmax(2).item()

        if next_token_id == EOS_IDX:
            break

        output_tokens.append(next_token_id)
        tgt_tensor = torch.tensor([[next_token_id]], dtype=torch.long).to(device)

    # Convert indices back to English words using vocab_en.lookup_token
    translated_sentence = ' '.join([vocab_en.lookup_token(token) for token in output_tokens])
    return translated_sentence


def score(input_data):
    """
    This function handles the incoming requests for scoring or translation.
    """
    try:
        input_json = json.loads(input_data)
        sentence = input_json.get('sentence', '')

        if not sentence:
            return json.dumps({'error': 'No sentence provided'})

        # Make sure models are loaded before running the translation
        load_model_and_config()

        translated_sentence = translate_sentence(sentence, device='cpu')
        if translated_sentence:
            return json.dumps({'translated_sentence': translated_sentence})
        else:
            return json.dumps({'error': 'Translation failed'})

    except Exception as e:
        return json.dumps({'error': str(e)})


# Example usage
if __name__ == "__main__":
    # Example input for testing
    input_data = json.dumps({"sentence": "Guten Morgen"})
    print(score(input_data))


Encoder model loaded successfully.
Decoder model loaded successfully.
Configuration data loaded successfully.
Model and configurations loaded successfully!
{"translated_sentence": "orange group - haired woman watches in the middle of a white ."}
